In [31]:
import pandas as pd

# Cargar los DataFrames (ajusta las rutas de los archivos a tu estructura)
df_obras = pd.read_csv(r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\corpus\escritos.csv")
df_dialogos = pd.read_csv(r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\corpus\dialogos.csv")
df_argumentos = pd.read_csv(r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\corpus\argumentos.csv")

In [32]:
df_obras.info()
df_dialogos.info()
df_argumentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   contenido  69 non-null     object
 1   titulo     69 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dialogo  35 non-null     object
 1   titulo   35 non-null     object
dtypes: object(2)
memory usage: 692.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   argumento  16 non-null     object
 1   titulo     16 non-null     object
dtypes: object(2)
memory usage: 388.0+ bytes


In [34]:
import spacy
nlp = spacy.load('es_dep_news_trf')

c:\Users\Lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy_transformers\layers\hf_shim.py:120: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self._model.

In [35]:
"""
Procesamiento de documentos pasa el texto por el pipline completo de spacy.

Incluye:
- Tokenizacion
- Etiquetado POS(part of speech)
- Analisis de dependencias
- Reconocimiento de entidades nombradas(NER)

Atributos principales:
- doc.ents las entidades nombradas reconocidas
- token.pos_ : La categoria gramatical de cada token
- token.dep_ : La relacion de dependencia sintactica
- token.head : El token del que depende el token actual
"""
# Función para procesar los diálogos y extraer entidades y temas
def procesar_dialogo(dialogo):
    doc = nlp(dialogo)
    
    # Extraer entidades
    entidades = [(ent.text, ent.label_) for ent in doc.ents]
    
    # Extraer sustantivos (posibles temas)
    temas = [token.text for token in doc if token.pos_ == "NOUN"]
    
    # Analizar las dependencias y relaciones
    dependencias = ["_".join([token.text, token.dep_, token.head.text]) for token in doc]
    
    return entidades, temas, dependencias

In [40]:
# Asegurarse de que las columnas no contengan valores nulos o vacíos antes de procesarlas
def limpiar_columna(columna):
    # Reemplazar valores nulos por cadena vacía
    columna = columna.fillna('')
    # Eliminar espacios al principio y al final de cada entrada
    columna = columna.apply(lambda x: x.strip())
    return columna

In [41]:
# Limpiar las columnas relevantes
df_dialogos['dialogo'] = limpiar_columna(df_dialogos['dialogo'])

In [ ]:
# Aplicar la función procesar_dialogo a la columna 'dialogo' de df_dialogos
df_dialogos['entidades'], df_dialogos['temas'], df_dialogos['dependencias'] = zip(*df_dialogos['dialogo'].apply(procesar_dialogo))

# Verificar las primeras filas para asegurarse de que las nuevas columnas están correctas
print(df_dialogos[['titulo', 'dialogo', 'entidades', 'temas', 'dependencias']].head())

In [ ]:
df_dialogos['entidades'] = limpiar_columna(df_dialogos['entidades'])
df_dialogos['temas'] = limpiar_columna(df_dialogos['temas'])
df_dialogos['dependencias'] = limpiar_columna(df_dialogos['dependencias'])

In [38]:
df_dialogos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dialogo  35 non-null     object
 1   titulo   35 non-null     object
dtypes: object(2)
memory usage: 692.0+ bytes


In [23]:
# Convertir las listas a texto (separado por espacios)
df_dialogos['entidades'] = df_dialogos['entidades'].apply(lambda x: ' '.join(x))
df_dialogos['temas'] = df_dialogos['temas'].apply(lambda x: ' '.join(x))
df_dialogos['dependencias'] = df_dialogos['dependencias'].apply(lambda x: ' '.join(x))

In [24]:
# Procesar todos los textos de las obras
df_obras['entidades'], df_obras['temas'], df_obras['dependencias'] = zip(*df_obras['contenido'].apply(procesar_dialogo))

# Mostrar los resultados procesados para las primeras obras
print(df_obras[['titulo', 'entidades', 'temas', 'dependencias']].head())

c:\Users\Lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


         titulo entidades                                              temas  \
0    Testamento        []  [testamento, mención, jardín, discípulos, test...   
1      Clitofon        []  [conversaciones, lecciones, sofista, discurso,...   
2  Definiciones        []  [definiciones, clases, interés, orden, doctrin...   
3        Cartas        []  [correspondencia, cartas, personajes, cartas, ...   
4   De lo justo        []  [comienzo, asunto, diálogo, pesar, ignorancia,...   

                                        dependencias  
0  [Este_det_testamento, testamento_nsubj_suyo, ,...  
1  [Clitofon_nsubj_defiende, ,_punct_acusado, acu...  
2  [Estas_det_definiciones, ciento_nummod_definic...  
3  [La_det_correspondencia, correspondencia_nsubj...  
4  [¿_punct_justo, Qué_nsubj_justo, es_cop_justo,...  


In [27]:
df_dialogos['dialogo'] = df_dialogos['dialogo'].astype(str)
df_dialogos['entidades'] = df_dialogos['entidades'].astype(str)
df_dialogos['temas'] = df_dialogos['temas'].astype(str)
df_dialogos['dependencias'] = df_dialogos['dependencias'].astype(str)

In [28]:
# Guardar los resultados procesados en archivos CSV

df_dialogos.to_csv("dialogos_procesados.csv", index=False)
df_argumentos.to_csv("argumentos_procesados.csv", index=False)
df_obras.to_csv("obras_procesadas.csv", index=False)

In [29]:
# Guardar los DataFrames en formato JSON
# se guarda como una lista de objetos (orient)
df_dialogos.to_json("dialogos_procesados.json", orient="records") 
df_argumentos.to_json("argumentos_procesados.json", orient="records")
df_obras.to_json("obras_procesadas.json", orient="records")